In [ ]:
import pandas as pd
df = pd.read_csv('mg.csv')

In [ ]:
# head function gives top 5 values and tail function gives last values,
# you can send argument too for these functions i.e head(20)/tail(20) for first 20 or last 20 records respectively
  
df.head() 
df.tail()

In [ ]:
# we also have attributes not called with brackets in the end lke we do for functions(head(), tail())

df.shape
df.columns
df.sample
df.dtypes

In [1]:
# Other functions.
# 1) nunique() `    --> GIve number of unique values in the column
# 2) unique()       --> GIve list of unique values in the column
# 3) value_counts() --> count how many times each unique value in the column
# 4) isna()         --> will give the list of rows having null value in the respective column.(concentrate the syntax)
# 5) isna().sum()   --> will give the sum of nan values.

df['Bedrooms'].nunique()
df['Bedrooms'].unique()
df['Bedrooms'].value_counts()
df[df['Bedrooms'].isna()]
df['Bedrooms'].isna().sum()

In [ ]:
# lets take 'Price' column and convert every value in to lakhs

df['price'] = df['Price'].str.split(' ').apply(lambda x: float(x[0]) * 100 if x[1] == 'Cr' else float(x[0]))

# what we are doing here, for example consider first value in data frame 50.5 lac:
# 1st part -->  df['Price'].str.split(' ') --> we are spliting 50.5 Lac : [50.5, lac]
# Then we are applying lambda function on this [50.5, lac]. here x[0] = 50.5 and x[1] = lac

# **************************************************************

# consider 4th value i.e 1.20 Cr, first we are spliting it to [1.20, Cr] 
# then we are applying lambda function to multiply by 100 if x[1] = 'Cr'



#  Other wise you can also do in three steps:
# 1) df['price'] = df['Price'].str.split(' ').str[0]
# 2) df['priceunit'] = df['Price'].str.split(' ').str[1]
# 3) df['price'] =df.apply(lambda row : row['price'] if row['priceunit'] == 'Lac' else row['price']*100, axis=1)

In [ ]:
#  Now let take 'Other' and extract 
# 'super area'/'column area' in to 'areatype' column 
# 'sqft'/'sqyrd' in to 'areasize' column

df['areasize'] = df['Other'].str.extract(r'\*(\d+)\s+')
df['areatype'] = df['Other'].str.extract(r'(carpet area|super area)')
df['areaunit'] = df['Other'].str.extract(r'(sqft|sqyrd)')

In [ ]:
# now lets convert every value  to sqft
# step1) convert data type of df['areasize'] -- > df1['areasize'] = pd.to_numeric(df1['areasize'])
# step2) then apply lambda function on df1 to take each row and multiply 'areasize' by 9 if 'areaunit' is sqyrd else multiply by 1.

df['areasize'] = pd.to_numeric(df['areasize'])
df['areasize'] = df.apply(lambda row: row['areasize']* (9 if row['areaunit'] == 'sqyrd' else 1), axis=1)

In [ ]:
# now lets split locality in to column area and take the requrired columns

df['area'] = df['Locality'].str.split(',').str[0]
df = df[['area', 'areasize', 'areatype', 'Bedrooms', 'price']]

In [ ]:
#  save the file. if index = false it will not take index values in to file

df.to_csv('workdata.csv', index = False)

In [2]:
# Now lets look in to few other functions ( copy paste each funtion to check the output)
# 1) df.loc[10:20]             --> Will give rows from 10 to 20
# 2) df.loc[10:20, 'area']    --> will give 10 to 20 rows of area colmn
# 3) df.loc[10:20, ['Price', 'Bedrooms']] --> will give 10 to 20 rows of area colmn
# 4) df.loc[df['area'] == 'Mallampet']   --> will give rows having mallampet values
# 5) df.loc[(df['Locality'] == 'Attapur') & (df['Bedrooms'] > 3),'Price']


In [ ]:
# lets clean the data

# find and remove duplicate values
df[df.duplicated()]
df.duplicated().sum()
df.drop_duplicates(inplace=True)

# lets drop values, dropna will remove all null values. 
df.dropna( inplace = True)
df.dropna(subset = ['area'], inplace = True)
df.dropna(subset = ['areasize'], inplace = True)

In [ ]:
#  working with Bedroom column 

df.dtypes

#  Lets also try to convert Bedrooms to int. 
df['Bedrooms'] = df['Bedrooms'].interpolate()
df['Bedrooms'] = pd.to_numeric(df['Bedrooms'])

#  It will through the error. 

df['Bedrooms'].unique()

# When you exicute above statement you will notice that there is different kind of object. 

# so first we have to deal with these types of data. 'coerce' will help convert those things in to nan values. 

df.loc[df['Bedrooms'] == '> 10'] = 11
df['Bedrooms'] = pd.to_numeric(df['Bedrooms'],  errors='coerce')



In [ ]:
# now lets see some stats 
df1.describe()

In [ ]:
# now lets see about groupy function
g1 = df1.groupby('areatype')
carpet = g1.get_group('carpet area')
super = g1.get_group('super area')
carpet
super
carpet.price
carpet.price.mean()


g = df1.groupby('area')
g1 = g.get_group('Mallampet')
g1.mean()


In [ ]:
# mean absolute deviation or standard deviation

import numpy as np
samples = np.random.normal(loc=0, scale=10, size=1000)
df = pd.DataFrame(samples, columns=['data'])
import matplotlib.pyplot as plt
plt.hist(df['data'], bins=20,  rwidth=0.8)
plt.title('Price Distribution')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.show()
mean = df.data.mean()
mean
std_deviation = df.data.std()
std_deviation
low = mean-3*std_deviation
high = mean+3*std_deviation
df[(df.data <low) | (df.data > high)]
df1 = df[(df.data>low)&(df.data<high)]

In [ ]:
# two famous methods to remove outliners

# IQR method or Tukey's method:


Q1 = df.price.quantile(0.25)
Q3 = df.price.quantile(0.75)
Q1, Q3

IQR = Q3 - Q1
IQR

low = Q1 - 1.5*IQR
high = Q3 + 1.5*IQR
low, high

df[(df.price <low) | (df.price > high)]
df1 = df[(df.data>low)&(df.data<high)]

In [ ]:
# zscore outline removal
df['zscore'] = ( df.price - df.price.mean() ) / df.price.std()
df[df['zscore']<-3]
df[df['zscore']> 3]
df1 = df[(df.zscore < 3) & (df.zscore > -3) ]

In [ ]:
# now lets see how a function is used on multiple groups, this is not necessary but just to understand how this concept works
def price_per_sqft(group):
    group['price_per_sqft'] = (group['price'] / group['areasize'])
    return group

carpetprice = carpet.groupby('price').apply(price_per_sqft)
superprice = super.groupby('price').apply(price_per_sqft)
superprice[superprice['area'] == 'Pragathi Nagar']
carpet.apply(lambda x: x.isnull().sum())
superprice[superprice['area'] == 'Pragathi Nagar'].max()

In [ ]:
# Lets see how to merge dataframes

maths = {'Student': ['John', 'Emily', 'Adam', 'Susan', 'Mike'],
        'maths_Marks': [85, 92, 78, 90, 87]}

maths = pd.DataFrame(maths)

science = pd.DataFrame({'Student': ['John', 'Emily', 'Adam', 'Susan', 'Mike'],
                'science_Marks': [80, 88, 82, 92, 85]})

df3 = pd.merge(maths, science, on="Student")

# how argument, lets add laresh and taresh, it is default inner
df3 = pd.merge(maths, science, on="Student", how = 'outer' )
df3 = pd.merge(maths, science, on="Student", how = 'right' )
df3 = pd.merge(maths, science, on="Student", how = 'left' )

# concatinate
roll_num = pd.Series([2,3,4,5,6], name = 'roll number')
df = pd.concat([maths, roll_num],axis=1)

# melt function

df4 = pd.melt(df3, id_vars=["Student"],var_name='subject', value_name='marks')


In [ ]:
# Lets plot few graphs
import matplotlib.pyplot as plt
plt.hist(df['price'], bins=20,  rwidth=0.8)
plt.title('Price Distribution')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.show()

# standard diviation -/+1 standard deviation 68.3%
# standard diviation -/+2 standard deviation 95.5%